# GPS, Conditioning, and Nonlinear Least Squares

Author: Alejandro C. Parra Garcia

In [ ]:
# import library
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d
import time

In [ ]:
# Constant
c_light=299792.458
earth_radius=6370

## Problem 1

### Functions

In [ ]:
#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
def plot_3d_Variable(df):
    x='x'
    y='y'
    z='z'
    cl=np.unique(df['hue'].values)
    clusterN=[]
    for i in cl:
        clusterN.append(df[df["hue"] == i])
        
    data=[]
    for n, cluster in enumerate(clusterN):
        name=cluster.values[0][3]
        opacity =1
        if(name=='earth surface'):
            opacity =0.01
        trace = go.Scatter3d(x = cluster[x], y = cluster[y], z = cluster[z],
                           mode = "markers", name = name,opacity=opacity,text = None)
        data.append(trace)
    
    
    title = "Position of the satelite and the gps"
    
    layout = go.Layout(
        title = title,
        scene = dict(
            xaxis = dict(title=x,ticklen= 5,zeroline= False),
            yaxis = dict(title=y,ticklen= 5,zeroline= False),
            zaxis = dict(title=z,ticklen= 5,zeroline= False),
        )
    )
    
    fig = dict(data = data, layout = layout)
    iplot(fig)

In [ ]:
# calculate 1 eq for the given values
def eq(a,b,c,t,x,y,z,d):
    return math.sqrt((x-a)**2+(y-b)**2+(z-c)**2)-c_light*(t-d)
    
# calculate the system of equations
def gps_f_x(x,s1,s2,s3,s4,t):
    sol=[]
    sol.append(eq(s1[0],s1[1],s1[2],t[0],x[0],x[1],x[2],x[3]))# first eq for satelate S1
    sol.append(eq(s2[0],s2[1],s2[2],t[1],x[0],x[1],x[2],x[3]))# first eq for satelate S2
    sol.append(eq(s3[0],s3[1],s3[2],t[2],x[0],x[1],x[2],x[3]))# first eq for satelate S3
    sol.append(eq(s4[0],s4[1],s4[2],t[3],x[0],x[1],x[2],x[3]))# first eq for satelate S4
    return sol

In [ ]:
def df_x(si,t,x):
    return (x[0]-si[0])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def df_y(si,t,x):
    return (x[1]-si[1])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def df_z(si,t,x):
    return (x[2]-si[2])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def df_d(si,t,x):
    return c_light

# Calculate the Jacobian Matrix of Partial Derivatives for x
def gps_df_x(x,s1,s2,s3,s4,t):
    A = np.array([[0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0]])
    satellites=[s1,s2,s3,s4]
    for i in range(4):
        A[i,0]=df_x(satellites[i],t[i],x)
        A[i,1]=df_y(satellites[i],t[i],x)
        A[i,2]=df_z(satellites[i],t[i],x)
        A[i,3]=df_d(satellites[i],t[i],x)
    return A

In [ ]:
def multivar_newton(x0,s1,s2,s3,s4,t,k=50,tol=1e-5,verbose=False):
    x=x0
    for i in range(k):
        
        f=gps_f_x(x,s1,s2,s3,s4,t)
        df=gps_df_x(x,s1,s2,s3,s4,t)
        
        if(all(abs(j) < tol for j in f)):
            # the tol level has been reach
            return x
        
        if(verbose):
            print("iter: "+str(i))
        # Solve the linear system instead of doing the inverse of the matrix
        s = np.linalg.solve(df, f)
        x=x-s
        if(verbose):
            print("x="+str(x[0])+" y="+str(x[1])+" z="+str(x[2])+", and d="+str(x[3]))

    return x   

In [ ]:
# create points in the earth surface
def add_earth_points(n,m):
    for i in range(n):
        step_n = 1/(n)
        point_n = i*step_n
        theta = point_n*2*math.pi
        
        for j in range(m):
            step_m=1/(m)
            point_m = j*step_m
            phi = point_m*2*math.pi
            
            A=earth_radius*math.cos(phi)*math.cos(theta)
            B=earth_radius*math.cos(phi)*math.sin(theta)
            C=earth_radius*math.sin(phi)
            df.loc[len(df.index)] = [A, B, C,'earth surface']
            
            # To see an evolution of the points of the earth surface being created uncomment the following code
            # Better to run with an small n and m. Otherwise is going to lag and posible crash due to ram problems
            # uncomment the following 4 lines and run this cell
            #plot_3d_Variable(df) 
#df = pd.DataFrame(columns=('x', 'y', 'z', 'hue'))
#df.loc[len(df.index)] = [0, 0, 0,'earth center']
#add_earth_points(4,4)

# If the points on the earth surface are not shown, change the opacity in the plot_3d_Variable method to a higher value

### main execution

In [ ]:
s1=[15600,7540,20140]
s2=[18760,2750,18610]
s3=[17610,14630,13480]
s4=[19170,610,18390]
t=[0.07074,0.07220,0.07690,0.07242]
x0=[0,0,6370,0]

# Create a DF to save the position of the satellites, the earth center, the earth surface, and the final position of the gps
df = pd.DataFrame(columns=('x', 'y', 'z', 'hue'))
ll=[s1,s2,s3,s4]
for i in range(len(ll)):
    # add the satelite position
    df.loc[i] = [ll[i][0],ll[i][1],ll[i][2],'satelite']
# add the center of the earth
df.loc[len(df.index)] = [0, 0, 0,'earth center']
# add points of earth location
add_earth_points(50,50)


# calculate the position of the gps
position=multivar_newton(x0,s1,s2,s3,s4,t,k=10,tol=1e-6,verbose=True)

# add the position of the gps to the dataframe
df.loc[len(df.index)] = [position[0], position[1],position[2],'gps']
# print the results
print("")
print("(x,y,z) = ("+str(position[0])+", "+str(position[1])+", "+str(position[2])+"), and d= "+str(position[3]))

In [ ]:
# print the dataframe
df

**you may have to re-execute the following cell for the image to load**

In [ ]:
# Plot the position of the satellites, the gps, and the earth
plot_3d_Variable(df)

## Problem 4

In [ ]:
import random

In [ ]:
# Constant
p=26570


### Functions

In [ ]:
list_tt=[1,2,3,4,5,6,7,8]
sal_list=[]
for i in range(len(list_tt)):
    sal_list.append([0,0,0])
sal_list

In [ ]:
# base on the phi and theta values of each satellites and d, we calculate the satellites position, distance and t
def calculate_satellites_position(satellites_phi_theta, x):
    #s1=[0,0,0]
    #s2=[0,0,0]
    #s3=[0,0,0]
    #s4=[0,0,0]
    #sal_list=[s1,s2,s3,s4]
    sal_list=[]
    for i in range(len(satellites_phi_theta)):
        sal_list.append([0,0,0])
    t=[0 for i in range(len(satellites_phi_theta))] # Create vector of t
    
    
    
    for i, satellites_i in enumerate(satellites_phi_theta):
        s_i_A=p*math.cos(satellites_i[0])*math.cos(satellites_i[1])
        s_i_B=p*math.cos(satellites_i[0])*math.sin(satellites_i[1])
        s_i_C=p*math.sin(satellites_i[0])
        
        sal_list[i][0]=s_i_A
        sal_list[i][1]=s_i_B
        sal_list[i][2]=s_i_C
        
        # calculate radious
        r_i = math.sqrt((s_i_A)**2+(s_i_B)**2+(s_i_C-earth_radius)**2)
        
        #calculate time
        t_i=x[3] + (r_i/c_light)
        t[i] = t_i
        
    return sal_list, t

In [ ]:
# Calculate the dif in the position base on dif_t and also calculate the error magnification factor
def change_t(x, x0, ll, t, dif_t, k=10,tol=1e-6):
    
    # Aply the change to the times
    new_t = [t[i]+dif_t[i]for i in range(len(t))]
    
    # Aply the method
    position = multivar_newton(x0,ll[0],ll[1],ll[2],ll[3],new_t,k=k,tol=tol,verbose=False)
    
    # calculate the difference in position ||(dif_x, dif_y, dif_z)||∞
    max_dif_x = max([abs(x[i]-position[i]) for i in range(len(x))])
    
    # calculate the difference in time ||(dif_t_1, ..., dif_t_n)||∞
    max_dif_t = max([abs(dif_t[i]) for i in range(len(dif_t))])
    
    # calculate the error magnification factor
    error_magnification_factor= max_dif_x / ( c_light*max_dif_t)
    
    return max_dif_x, error_magnification_factor

In [ ]:
# Run an iterative proces to check multiple diferent times variables
def iterate_errors(x, x0, ll, t, s_i=5, s_j=5, s_n=5, s_m=5):
    df_errors = pd.DataFrame(columns=('dif_t0','dif_t1','dif_t2','dif_t3', 'distance (m)', 'error_magnification_factor'))
    
    dif_t=[-1e-8,-1e-8,-1e-8,-1e-8]
    step_i=0
    step_j=0
    step_n=0
    step_m=0
    
    if(s_i==1):
        dif_t[0]=0
    else:
        step_i=(1e-8*2)/(s_i-1)
    if(s_j==1):
        dif_t[1]=0
    else:
        step_j=(1e-8*2)/(s_j-1)
    if(s_n==1):
        dif_t[2]=0
    else:
        step_n=(1e-8*2)/(s_n-1)
    if(s_m==1):
        dif_t[3]=0
    else:
        step_m=(1e-8*2)/(s_m-1)
        
    for i in range(s_i):
        v_i=dif_t[0]+step_i*i
        for j in range(s_j):
            v_j=dif_t[1]+step_j*j
            for n in range(s_n):
                v_n=dif_t[2]+step_n*n
                for m in range(s_m):
                    v_m=dif_t[3]+step_m*m

                    if(v_i==0 and v_j==0 and v_n==0 and v_m==0):
                        print("A difference of [0,0,0,0] is skiped")
                        continue
                    
                    # Calculate the distance and Error magnification factor
                    dd, emf = change_t(x, x0, ll, t, [v_i,v_j,v_n,v_m])
                    
                    
                    #save the result
                    df_errors.loc[len(df_errors.index)] = [v_i, v_j, v_n, v_m, (dd*1000), emf]
    return df_errors

In [ ]:
# Plot the erros (It can only plot 2 of the 4 satellaties times dif_times) and the distance or EMF
# xx and yy are the names of 2 of the t's, "dif_t0", "dif_t1", "dif_t2" or "dif_t3" ...
# zz is the column of distance or the EMF
def plot_3d_Errors(df, xx, yy, zz):
    x=xx
    y=yy
    z=zz
    text=""
    if ( (len(df.columns)-2) == 4):
        # 4 sattelites
        text=["t: (" + str( df['dif_t0'][ind] ) + ", " + str(df['dif_t1'][ind]) + ", " + str(df['dif_t2'][ind]) + ", " + str(df['dif_t3'][ind]) +  ") distance="+str(round(df['distance (m)'][ind],4))+ ", EMF= "+str(round(df['error_magnification_factor'][ind],4)) for ind in df.index]
    elif ( (len(df.columns)-2) == 8):
        # 8 sattelites
        text=["t: (" + str( df['dif_t0'][ind] ) + ", " + str(df['dif_t1'][ind]) + ", " + str(df['dif_t2'][ind]) + ", " + str(df['dif_t3'][ind]) + ", " + str(df['dif_t4'][ind]) + ", " + str(df['dif_t5'][ind]) + ", " + str(df['dif_t6'][ind]) + ", " + str(df['dif_t7'][ind]) + ") distance="+str(round(df['distance (m)'][ind],4))+ ", EMF= "+str(round(df['error_magnification_factor'][ind],4)) for ind in df.index]
    else:
        text=["distance="+str(round(df['distance (m)'][ind],4))+ ", EMF= "+str(round(df['error_magnification_factor'][ind],4)) for ind in df.index]
    
    #print(text)
    data=[]
    trace = go.Scatter3d(x = df[x], y = df[y], z = df[z],
                           mode = "markers",text = text)
    data.append(trace)
    
    title = zz+" base on the diference of " + xx + " and " + yy + " (if result is a plane then the others t's constant at 0)"
    
    layout = go.Layout(
        title = title,
        scene = dict(
            xaxis = dict(title=x,ticklen= 5,zeroline= False),
            yaxis = dict(title=y,ticklen= 5,zeroline= False),
            zaxis = dict(title=z,ticklen= 5,zeroline= False),
        )
    )
    
    fig = dict(data = data, layout = layout)
    iplot(fig)


### main execution

In [ ]:
x=[0,0,6370,0.0001]
#x0=[-41.77270940460345, -16.789194039784554, 6370.059559201504, -0.0032015658289978126]
x0=[0, 6370, 0, 0]

#choose random phi and theta for each sattelite
random.seed(5)
s1_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s2_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s3_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s4_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]

In [ ]:
# Calculate the A,B,C and t for each sattelite
ll, t = calculate_satellites_position([s1_phi_theta,s2_phi_theta,s3_phi_theta,s4_phi_theta],x)

In [ ]:
# Save the information into the df
df = pd.DataFrame(columns=('x', 'y', 'z', 'hue'))
#ll=[s1,s2,s3,s4]
for i in range(len(ll)):
    # add the satelite position
    df.loc[i] = [ll[i][0],ll[i][1],ll[i][2],'satelite']
# add the center of the earth
df.loc[len(df.index)] = [0, 0, 0,'earth center']
# add points of earth location
add_earth_points(50,50)

In [ ]:
# calculate the position of the gps
position=multivar_newton(x0,ll[0],ll[1],ll[2],ll[3],t,k=10,tol=1e-6,verbose=True)

# add the position of the gps to the dataframe
df.loc[len(df.index)] = [position[0], position[1],position[2],'gps']
# print the results
print("")
print("(x,y,z) = ("+str(position[0])+", "+str(position[1])+", "+str(position[2])+"), and d= "+str(position[3]))

**you may have to re-execute the following cell for the image to load**

In [ ]:
# Plot the position of the satellites, the gps, and the earth
plot_3d_Variable(df)

#### Tring different variations of the 	dif_ti’s

In [ ]:
dif_t=[0,0,-1e-8,0]
change_t(x, x0, ll, t, dif_t, k=10,tol=1e-6)

In [ ]:
dif_t=[0,1e-8,0,0]
change_t(x, x0, ll, t, dif_t, k=10,tol=1e-6)

**Iterating the proces**

In [ ]:
# Evolution for t_2 and t_3
df_errors=iterate_errors(x, x0, ll, t,s_i=1, s_j=1, s_n=50, s_m=50)

In [ ]:
df_errors

In [ ]:
plot_3d_Errors(df_errors, xx="dif_t2", yy="dif_t3", zz="distance (m)")

In [ ]:
# Evolution for t_0 and t_1
df_errors=iterate_errors(x, x0, ll, t,s_i=50, s_j=50, s_n=1, s_m=1)
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t1", zz="distance (m)")

In [ ]:
# Evolution for t_0 and t_3
df_errors=iterate_errors(x, x0, ll, t,s_i=50, s_j=1, s_n=1, s_m=50)
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t3", zz="distance (m)")

In [ ]:
# Evolution 10.000 points for all the conbinations
df_errors=iterate_errors(x, x0, ll, t,s_i=10, s_j=10, s_n=10, s_m=10)
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t1", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t2", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t3", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t1", yy="dif_t2", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t1", yy="dif_t3", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t2", yy="dif_t3", zz="distance (m)")

In [ ]:
df_errors=iterate_errors(x, x0, ll, t,s_i=15, s_j=15, s_n=15, s_m=15)

In [ ]:
df_errors

In [ ]:
df_errors[df_errors['distance (m)'] == df_errors['distance (m)'].max()]

We can see that the maximum distance is 53m which causes an EMF of 17.69, this occurs when we modify the times in this way.

t0=1e-8

t1=1e-8

t2=-1e-8

t3=-1e-8

In [ ]:
df_errors['distance (m)'].hist(legend=True,figsize=(10,10))
df_errors['error_magnification_factor'].hist(legend=True,figsize=(10,10))

## Problem 5

### main execution

5º Correspond to less than 5 * (π/2)/90 --> π * (1/36)

In [ ]:
x=[0,0,6370,0.0001]
#x0=[-41.77270940460345, -16.789194039784554, 6370.059559201504, -0.0032015658289978126]
x0=[0, 6370, 0, 0]

#choose random phi and theta for each sattelite
random.seed(5)
s1_phi_theta=[random.random()*(math.pi/36)+(math.pi/2),random.random()*(math.pi/36)]
s2_phi_theta=[random.random()*(math.pi/36)+(math.pi/2),random.random()*(math.pi/36)]
s3_phi_theta=[random.random()*(math.pi/36)+(math.pi/2),random.random()*(math.pi/36)]
s4_phi_theta=[random.random()*(math.pi/36)+(math.pi/2),random.random()*(math.pi/36)]

In [ ]:
# Calculate the A,B,C and t for each sattelite
ll, t = calculate_satellites_position([s1_phi_theta,s2_phi_theta,s3_phi_theta,s4_phi_theta],x)

In [ ]:
# Save the information into the df
df = pd.DataFrame(columns=('x', 'y', 'z', 'hue'))
#ll=[s1,s2,s3,s4]
for i in range(len(ll)):
    # add the satelite position
    df.loc[i] = [ll[i][0],ll[i][1],ll[i][2],'satelite']
# add the center of the earth
df.loc[len(df.index)] = [0, 0, 0,'earth center']
# add points of earth location
add_earth_points(50,50)

In [ ]:
# calculate the position of the gps
position=multivar_newton(x0,ll[0],ll[1],ll[2],ll[3],t,k=10,tol=1e-6,verbose=True)

# add the position of the gps to the dataframe
df.loc[len(df.index)] = [position[0], position[1],position[2],'gps']
# print the results
print("")
print("(x,y,z) = ("+str(position[0])+", "+str(position[1])+", "+str(position[2])+"), and d= "+str(position[3]))

**you may have to re-execute the following cell for the image to load**

In [ ]:
# Plot the position of the satellites, the gps, and the earth
plot_3d_Variable(df)

#### Tring different variations of the 	dif_ti’s using the iterative process

In [ ]:
df_errors=iterate_errors(x, x0, ll, t,s_i=15, s_j=15, s_n=15, s_m=15)

In [ ]:
df_errors

In [ ]:
df_errors[df_errors['distance (m)'] == df_errors['distance (m)'].max()]

We can see that the maximum distance is 32985m which causes an EMF of 11002.78, this occurs when we modify the times in this way.

t0=-1e-8

t1=-1e-8

t2=1e-8

t3=-1e-8

In [ ]:
df_errors['distance (m)'].hist(legend=True,figsize=(10,10))
df_errors['error_magnification_factor'].hist(legend=True,figsize=(10,10))

## Problem 6

### functions


In [ ]:
# calculate 1 eq for the given values
def eq(a,b,c,t,x,y,z,d):
    return math.sqrt((x-a)**2+(y-b)**2+(z-c)**2)-c_light*(t-d)
    
# calculate the system of equations
def gps_r_x(x, s_i, t):
    sol=[]
    for count, value in enumerate(s_i):
        sol.append(eq(value[0], value[1], value[2], t[count],x[0],x[1],x[2],x[3]))# first eq for satelate S_i
    
    return sol

In [ ]:
def dr_x(si,t,x):
    return (x[0]-si[0])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def dr_y(si,t,x):
    return (x[1]-si[1])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def dr_z(si,t,x):
    return (x[2]-si[2])/(math.sqrt((x[0]-si[0])**2+(x[1]-si[1])**2+(x[2]-si[2])**2))
def dr_d(si,t,x):
    return c_light

# Calculate the Jacobian Matrix of Partial Derivatives for x
def gps_dr_x(x, s_i, t):
    A = np.zeros((len(s_i), len(x)))
    #A = np.array([[0.0,0.0,0.0,0.0],
    #             [0.0,0.0,0.0,0.0],
    #             [0.0,0.0,0.0,0.0],
    #             [0.0,0.0,0.0,0.0]])
    for i in range(len(s_i)):
        A[i,0]=df_x(s_i[i], t[i], x)
        A[i,1]=df_y(s_i[i], t[i], x)
        A[i,2]=df_z(s_i[i], t[i], x)
        A[i,3]=df_d(s_i[i], t[i], x)
    return A

In [ ]:
def gauss_Newton_iteration(x0, satellites_list, t, k=50, tol=1e-5, verbose=False):
    x=x0
    for i in range(k):
        
        r=gps_r_x(x, satellites_list, t)
        A=gps_dr_x(x, satellites_list, t)
        
        if(all(abs(j) < tol for j in r)):
            # the tol level has been reach
            return x
        
        if(verbose):
            print("iter: "+str(i))
            
        # Solve for s
        # A_T * A * s = A_T * r  --> A = Q*R
        # (Q*R)_T * Q*R * s=(Q*R)_T * r
        # R_T * Q_T * Q * R * s = R_T * Q_T * r  Since Q_T = Q_-1 --> Q_T * Q = Q_-1 * Q = 1
        # R_T * R * s = R_T * Q_T * r --> p = Q_T * r 
        # R * s = p
        
        Q, R = np.linalg.qr(A)  # A = Q*R
        p = np.dot(Q.T, r)      # p = Q_T * r 
            
            
        # Solve the linear system instead of doing the inverse of the matrix
        s = np.linalg.solve(R, p) # R * s = p
        x = x - s
        
        if(verbose):
            print("x="+str(x[0])+" y="+str(x[1])+" z="+str(x[2])+", and d="+str(x[3]))

    return x   
    

In [ ]:
# Calculate the dif in the position base on dif_t and also calculate the error magnification factor
def change_t_8S(x, x0, ll, t, dif_t, k=10,tol=1e-6):
    
    # Aply the change to the times
    new_t = [t[i]+dif_t[i]for i in range(len(t))]
    
    # Aply the method
    position = gauss_Newton_iteration(x0, ll, new_t, k=k, tol=tol, verbose=False)
    
    # calculate the difference in position ||(dif_x, dif_y, dif_z)||∞
    max_dif_x = max([abs(x[i]-position[i]) for i in range(len(x))])
    
    # calculate the difference in time ||(dif_t_1, ..., dif_t_n)||∞
    max_dif_t = max([abs(dif_t[i]) for i in range(len(dif_t))])
    
    # calculate the error magnification factor
    error_magnification_factor= max_dif_x / ( c_light*max_dif_t)
    
    return max_dif_x, error_magnification_factor

In [ ]:
# Run an iterative proces to check multiple diferent times variables for 8 sattelites
def iterate_errors_8S(x, x0, ll, t, s_i=2, s_j=2, s_n=2, s_m=2, s_a=2, s_b=2, s_c=2, s_d=2):
    df_errors = pd.DataFrame(columns=('dif_t0','dif_t1','dif_t2','dif_t3','dif_t4','dif_t5','dif_t6','dif_t7', 'distance (m)', 'error_magnification_factor'))
    
    dif_t=[-1e-8 for i in range(len(ll))] # Create inicial vector of -1e-8 t for each satelite
    step_i=0
    step_j=0
    step_n=0
    step_m=0
    step_a=0
    step_b=0
    step_c=0
    step_d=0
    
    if(s_i==1):
        dif_t[0]=0
    else:
        step_i=(1e-8*2)/(s_i-1)
    if(s_j==1):
        dif_t[1]=0
    else:
        step_j=(1e-8*2)/(s_j-1)
    if(s_n==1):
        dif_t[2]=0
    else:
        step_n=(1e-8*2)/(s_n-1)
    if(s_m==1):
        dif_t[3]=0
    else:
        step_m=(1e-8*2)/(s_m-1)
    
    if(s_a==1):
        dif_t[4]=0
    else:
        step_a=(1e-8*2)/(s_a-1)
    if(s_b==1):
        dif_t[5]=0
    else:
        step_b=(1e-8*2)/(s_b-1)
    if(s_c==1):
        dif_t[6]=0
    else:
        step_c=(1e-8*2)/(s_c-1)
    if(s_d==1):
        dif_t[7]=0
    else:
        step_d=(1e-8*2)/(s_d-1)
       
    # Iterate all the sattelites  
    for i in range(s_i):
        v_i=dif_t[0]+step_i*i
        for j in range(s_j):
            v_j=dif_t[1]+step_j*j
            for n in range(s_n):
                v_n=dif_t[2]+step_n*n
                for m in range(s_m):
                    v_m=dif_t[3]+step_m*m

                    for a in range(s_a):
                        v_a=dif_t[4]+step_a*a
                        for b in range(s_b):
                            v_b=dif_t[5]+step_b*b
                            for c in range(s_c):
                                v_c=dif_t[6]+step_c*c
                                for d in range(s_d):
                                    v_d=dif_t[7]+step_d*d
                                    
                                    
                                    if(v_i==0 and v_j==0 and v_n==0 and v_m==0 and v_a==0 and v_b==0 and v_c==0 and v_d==0):
                                        print("A difference of [0,0,0,0,0,0,0,0] is skiped")
                                        continue
                    
                                    # Calculate the distance and Error magnification factor
                                    dd, emf = change_t_8S(x, x0, ll, t, [v_i,v_j,v_n,v_m, v_a,v_b,v_c,v_d])
                    
                    
                                    #save the result  the distance in meters
                                    df_errors.loc[len(df_errors.index)] = [v_i, v_j, v_n, v_m, v_a, v_b, v_c, v_d, (dd*1000), emf]
    return df_errors

### main execution

In [ ]:
x=[0,0,6370,0.0001]

x0=[0, 0, 0, 0]

#choose random phi and theta for each sattelite
random.seed(5)
s1_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s2_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s3_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s4_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s5_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s6_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s7_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]
s8_phi_theta=[random.random()*(math.pi/2),random.random()*(2*math.pi)]

In [ ]:
# Calculate the A,B,C and t for each sattelite
ll, t = calculate_satellites_position([s1_phi_theta,s2_phi_theta,s3_phi_theta,s4_phi_theta,s5_phi_theta,s6_phi_theta,s7_phi_theta,s8_phi_theta],x)

In [ ]:
# Save the information into the df
df = pd.DataFrame(columns=('x', 'y', 'z', 'hue'))
#ll=[s1,s2,s3,s4]
for i in range(len(ll)):
    # add the satelite position
    df.loc[i] = [ll[i][0],ll[i][1],ll[i][2],'satelite']
# add the center of the earth
df.loc[len(df.index)] = [0, 0, 0,'earth center']
# add points of earth location
add_earth_points(50,50)

In [ ]:
# calculate the position of the gps
position=gauss_Newton_iteration(x0, ll, t, k=10, tol=1e-6, verbose=True)

# add the position of the gps to the dataframe
df.loc[len(df.index)] = [position[0], position[1],position[2],'gps']
# print the results
print("")
print("(x,y,z) = ("+str(position[0])+", "+str(position[1])+", "+str(position[2])+"), and d= "+str(position[3]))

**you may have to re-execute the following cell for the image to load**

In [ ]:
# Plot the position of the satellites, the gps, and the earth
plot_3d_Variable(df)

#### Tring different variations of the 	dif_ti’s using the iterative process

In [ ]:
df_errors=iterate_errors_8S(x, x0, ll, t,s_i=4, s_j=4, s_n=4, s_m=4, s_a=4, s_b=4, s_c=4, s_d=4)

In [ ]:
df_errors

In [ ]:
# Plot some of the combinations (In total there are 5,040 combinations of 2 different sattelites, so im not ploting all)

plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t1", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t2", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t0", yy="dif_t3", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t1", yy="dif_t2", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t1", yy="dif_t3", zz="distance (m)")
plot_3d_Errors(df_errors, xx="dif_t2", yy="dif_t3", zz="distance (m)")

In [ ]:
df_errors[df_errors['distance (m)'] == df_errors['distance (m)'].max()]

We can see that the maximum distance is 13.5m which causes an EMF of 4.5, this occurs when we modify the times in this way.

t0=-1e-8

t1=-1e-8

t2=-1e-8

t3=-1e-8

t4=1e-8

t5=1e-8

t6=-1e-8

t7=1e-8

In [ ]:
df_errors['distance (m)'].hist(legend=True,figsize=(10,10))
df_errors['error_magnification_factor'].hist(legend=True,figsize=(10,10))

## Final thoughts

A good initial vector could be the center of the earth at 0,0,0 since it is the closer point to all the points in the surface.


We can see from the results that the configuration with 8 satellites achieve a higher precision when the t's are modified, since the maximum distance was 13.5m with an EMF of only 4.5. Compared to the used of 4 unbunched satellites where the maximum distance was 53 meters with an EMF of 17.69 in this case the EMG was almost 4 times bigger. But if we go to the last case, where the 4 satellites were bunched together, we can see that the maximum distance grows to 32,985 meters, almost 33 Km and the EMF is 11,002.78 in this case the system is extremely week, as an slight change in the times produces errors of multiple kilometers.


Base on this information the best model is the one where the receiver has access to the maximum number of satellites possible, and also it is extremely important that those satellites are not bunched together as this decrease the accuracy by a lot.
